# Librairies nécessaires

In [ ]:

import pandas as pd 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import collections
import glob
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score,classification_report, hamming_loss

In [ ]:
#Chargement des données (Ici données confidentiels donc pas sur Github)
df= pd.read_csv('challenge_filtered.csv') # Remplacer par le nom ou le chemin du fichier

# Data Exploration / Data Visualization

In [ ]:
'''labels=df.iloc[ : , 3:54].columns
fig = plt.figure(figsize=(20,6))
ax = fig.add_axes([0,0,1,1])
total_count = []
for label in labels:
    total_count.append(len(df[df[label] == 1]))
ax.bar(labels,total_count, color=['red', 'green', 'blue', 'purple', 'orange', 'yellow'])
for i,data in enumerate(total_count):
    plt.text(i-.25, 
              data/total_count[i]+100, 
              total_count[i], 
              fontsize=5)
plt.title('Number of comments per label')
plt.xlabel('Labels')
plt.ylabel('Number of comments')
plt.xticks(rotation=45)
plt.show()'''


In [ ]:
class BarPlot:
    
    def __init__(self, df, color):
        """
        Initialise la classe avec un dataframe et une couleur pour le graphique.
        """
        self.df = df
        self.color = color
        
    def plot(self):
        """
        Trace un graphique en barres à partir du dataframe et de la couleur donnés en entrée.
        """
        labels = self.df.iloc[:, 2:56].columns # On récupère nos labels

        fig = plt.figure(figsize=(12, 6))
        ax = fig.add_axes([0, 0, 1, 1])
        total_count = []
        for label in labels:
            total_count.append(len(self.df[self.df[label] == 1]))

        ax.bar(labels, total_count, color=self.color)

        #plt.axhline(y=80, color='brown', linestyle='dotted') # Add brown dotted line at y=80

        # Add labels to the horizontal line
        #ax.text(58, 90, 'Aprés augmentation', fontsize=10, color='brown', ha='left', va='center')
        plt.title('Nombre de comptes rendus par codes')
        plt.xlabel('Labels')
        plt.ylabel('Nombre de comptes rendus')
        ax.set_xticklabels([]) # Enlever les étiquettes sur l'axe x
        plt.show()



In [ ]:
def create_wordcloud(df):
    # Concaténer tous les textes dans une seule chaîne de caractères
    texte = " ".join(df["full_text"])

    # Créer un nuage de mots
    nuage_mot = WordCloud(width=800, height=400, background_color="white").generate(texte)

    # Afficher le nuage de mots
    plt.imshow(nuage_mot)
    plt.axis("off")
    plt.show()

#  Preprocessing | splitting

In [ ]:

def preprocess(text):
    # Convertir le texte en minuscule
    text = text.lower()
    
    # Supprimer les chiffres qui ne sont pas collés à des lettres
    text = re.sub(r'\b\d+\b', '', text)
    
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]|_', '', text)
    
    # Tokenization des mots
    tokens = word_tokenize(text)
    
    # Supprimer les stopwords
    tokens = [token for token in tokens if not token in stopwords.words('english')]
    
    # Lemmatisation des mots
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Rejoindre les tokens pour former une chaîne de caractères
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# On applique la fonction à nos comptes rendus
df['full_text'] = df['full_text'].apply(preprocess)

In [ ]:
# On récupère la colonne contenant le texte des comptes rendus médicaux
X = df['full_text']

# On récupère toutes les colonnes correspondant aux codes des diagnostics
y = df.drop(["Patient_Id", "Admission_Id", "age", "gender", "ethnicity"], axis=1) 

# On utilise la méthode MultilabelStratifiedKFold pour diviser les données 
# tout en veillant à préserver la proportion de chaque classe dans chaque set de données.
mskf = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=0)

# On récupère les index des lignes des données qui seront utilisées pour l'entraînement et les tests
train_index, test_index = next(mskf.split(X, y))

# On récupère les données 
X_train, y_train = X.iloc[train_index], y.iloc[train_index]
X_test, y_test = X.iloc[test_index], y.iloc[test_index]

# On fusionne les données  en une seule dataframe
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# On écrit les données d'entraînement dans un fichier CSV
train_df.to_csv('train_data.csv', index=False)

# On écrit les données de test dans un fichier CSV
test_df.to_csv('test_data.csv', index=False)


# Models

Random Forest | Paramétres : max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300 (Choisi par GridSearch)

In [ ]:
labels=['427.31', '428.0', '414.01', '412', '410.71', '424.0','427.89', '424.1','427.1', '411.1', '425.4', '414.00', 
'427.5','428.33', '427.32', '428.32', '428.23', '428.22', '413.9', '414.8',
'397.0', '428.30', '427.41', '410.41', '410.11', '426.0', '428.21',
'398.91', '414.02', '423.9', '421.0', '428.31', '396.2', '427.81',
'428.20', '396.3', '410.91', '428.43', '414.2', '410.72', '426.3',
'423.3', '411.89', '424.2', '425.1', '428.42', '426.4', '427.0',
'420.90', '429.9', '410.01', '426.11', '426.13', '410.31']
class RFModel:
    
    def __init__(self, X_train, X_test, y_train, y_test):
        """
        Initialise le modèle de forêt aléatoire avec les données d'entraînement et de test.
        
        Args:
            X_train (array-like): Données d'entraînement.
            X_test (array-like): Données de test.
            y_train (array-like): Labels des données d'entraînement.
            y_test (array-like): Labels des données de test.
        """
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.vectorizer = TfidfVectorizer()
        self.rf = OneVsRestClassifier(RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300))
    
    def train(self):
        """
        Entraîne le modèle de forêt aléatoire sur les données d'entraînement.
        """
        # Vectorisation des données d'entraînement
        X_train_vectorized = self.vectorizer.fit_transform(self.X_train)
        # Entraînement du modèle sur les données d'entraînement
        self.rf.fit(X_train_vectorized, self.y_train)
    
    def evaluate(self):
        """
        Évalue le modèle sur les données de test.
        """
        # Vectorisation des données de test
        X_test_vectorized = self.vectorizer.transform(self.X_test)
        # Prédiction des labels sur les données de test
        y_pred = self.rf.predict(X_test_vectorized)
        # Calcul des métriques de performance
        recall = recall_score(self.y_test, y_pred, average='macro')
        f1 = f1_score(self.y_test, y_pred, average='macro')
        auc = roc_auc_score(self.y_test, y_pred, average='weighted')
        hamming = hamming_loss(self.y_test, y_pred)
        # Affichage des résultats
        CR = classification_report(self.y_test, y_pred, target_names=labels)
        print("Recall: {:.3f}".format(recall))
        print("F1-score: {:.3f}".format(f1))
        print("AUC: {:.3f}".format(auc))
        print("Hamming Loss: {:.3f}".format(hamming))
        print(CR)


SVM| Paramétres : Kernel : linear

In [ ]:
class SVMModel:
    
    def __init__(self, X_train, X_test, y_train, y_test):
        """
        Initialise le modèle de forêt aléatoire avec les données d'entraînement et de test.
        
        Args:
            X_train (array-like): Données d'entraînement.
            X_test (array-like): Données de test.
            y_train (array-like): Labels des données d'entraînement.
            y_test (array-like): Labels des données de test.
        """
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.vectorizer = TfidfVectorizer()
        self.svm = OneVsRestClassifier(SVC(kernel='linear'))
    
    def train(self):
        """
        Entraîne le modèle de forêt aléatoire sur les données d'entraînement.
        """
        # Vectorisation des données d'entraînement
        X_train_vectorized = self.vectorizer.fit_transform(self.X_train)
        # Entraînement du modèle sur les données d'entraînement
        self.svm.fit(X_train_vectorized, self.y_train)
    
    def evaluate(self):
        """
        Évalue le modèle sur les données de test.
        """
        # Vectorisation des données de test
        X_test_vectorized = self.vectorizer.transform(self.X_test)
        # Prédiction des labels sur les données de test
        y_pred = self.svm.predict(X_test_vectorized)
        # Calcul des métriques de performance
        precision = precision_score(self.y_test, y_pred, average='macro')
        recall = recall_score(self.y_test, y_pred, average='macro')
        f1 = f1_score(self.y_test, y_pred, average='macro')
        auc = roc_auc_score(self.y_test, y_pred, average='macro')
        hamming = hamming_loss(self.y_test, y_pred)
        # Affichage des résultats
        CR = classification_report(self.y_test, y_pred, target_names=labels)
        print("Precision: {:.3f}".format(precision))
        print("Recall: {:.3f}".format(recall))
        print("F1-score: {:.3f}".format(f1))
        print("AUC: {:.3f}".format(auc))
        print("Hamming Loss: {:.3f}".format(hamming))
        print(CR)


In [ ]:
import matplotlib.pyplot as plt

class BarPlot2:
    
    def __init__(self, df, color):
        """
        Initialise la classe avec un dataframe et une couleur pour le graphique.
        """
        self.df = df
        self.color = color
        
    def plot(self):
        """
        Trace un graphique en barres à partir du dataframe et de la couleur donnés en entrée.
        """
        labels = self.df.iloc[:, 2:56].columns # On récupère nos labels

        fig = plt.figure(figsize=(12, 6))
        ax = fig.add_axes([0, 0, 1, 1])
        total_count = []
        for label in labels:
            count = len(self.df[self.df[label] == 1])
            if count < 80:
                count = count + (80 - count) # Add up to 80 for counts less than 80
            total_count.append(count)

        # Color the bars with counts less than 80 in the specified color, and color the bars with augmented counts in blue
        colors = [self.color if count < 80 else 'blue' for count in total_count]

        ax.bar(labels, total_count, color=colors)

        # Add labels to the horizontal line
        
        plt.title('Nombre de comptes rendus par codes')
        plt.xlabel('Labels')
        plt.ylabel('Nombre de comptes rendus')
        ax.set_xticklabels([]) # Enlever les étiquettes sur l'axe x
        plt.show()




        
